In [ ]:
# CNN


In [9]:
# IMPORTS
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.datasets import mnist

In [2]:
# LOAD DATA
dataframe = pd.read_csv('ML\data\dynamic_pus_using_pus50000_15PUs_201910_2616_25.txt', delimiter=',', header=None)
dataframe_max = pd.read_csv('ML\data\dynamic_pus_max_power50000_15PUs_201910_2616_25.txt', delimiter=',', header=None)

dataframe.reset_index(drop=True, inplace=True)
dataframe_max.reset_index(drop=True, inplace=True)

dataframe_tot = pd.concat([dataframe, dataframe_max.iloc[:, dataframe_max.columns.values[-1]]], axis=1,
                        ignore_index=True)
idx = dataframe_tot[dataframe_tot[dataframe_tot.columns[-1]] == -float('inf')].index
dataframe_tot.drop(idx, inplace=True)

data_reg = np.concatenate((dataframe_tot.values[:, 0:dataframe_tot.shape[1]-3], 
                           dataframe_tot.values[:, dataframe_tot.shape[1]-1:dataframe_tot.shape[1]]), axis=1)
data_class = dataframe_tot.values[:, 0:dataframe_tot.shape[1]-1]
y_class_power = dataframe_tot.values[:, -1]
del dataframe, dataframe_max

In [41]:
# INIT
number_samples = [5] + list(range(10, 101, 10)) + [120, 150, 200, 250, 300, 400, 500, 700] + list(range(1000, 4001, 1000))
number_samples = [15]

In [28]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [13]:
x_train.reshape((60000, 28, 28, 1))

array([[[[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        ...,

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]]],


       [[[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        ...,

        [[0],
         [0],
         [0],
         ...,
         [0],


In [18]:
a = [[-90] * 4000 for _ in range(4000)]

In [31]:
num_pus = (data_reg.shape[1] - 3)//3

In [27]:
tf.convert_to_tensor(a)

<tf.Tensor 'Const:0' shape=(4000, 4000) dtype=int32>

In [30]:
type(x_train)

numpy.ndarray

In [32]:
num_pus

15

In [35]:
for pu_n in range(num_pus):
    x, y, p = max(0, int(data_reg[0][pu_n*3])), min(3999, int(data_reg[0][pu_n*3+1])), data_reg[0][pu_n*3+2]
    a[x][y] = p

In [44]:
bb = np.array([a, a], dtype=float)

In [47]:
bb.shape

(2, 4000, 4000)

In [ ]:
## CNN
average_class_diff_power = []
fp_mean_power = []
validation_size = 0.33
for number_sample in number_samples:
    sample = math.ceil(number_sample * (1 + validation_size))
    